In [1]:
import pandas as pd
from parse_hh_data import download, parse
import ast
from bs4 import BeautifulSoup
from datetime import datetime
import re

In [2]:
import nltk
#nltk.download("stopwords")

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [3]:
import pandas as pd
import re
from nltk.stem.snowball import SnowballStemmer
#import stop_words

In [4]:
df = pd.read_csv("resumes_step1.csv", encoding = 'utf-16')

In [5]:
df.head(3)

,birth_date,gender,area,education,title,education_level,salary.amount,salary.currency,specialization,skill_set,skills_desc,exp_length,exp_description
0,27 октября 1954,Мужчина,Москва,"[{'year': 1976, 'name': 'Высшее Военное Инжене...","Организатор корпоративных охот, рыбалок и эко-...",Высшее образование,60000.0,руб.,Руководитель проектов,"Работа в Фондах, НКО и ТПП, Союзах и Ассоциаци...",В работе: Аналитик.Трудоголик. Хобби: 50 лет ...,10915.0,Руководство административно-хозяйственной деят...
1,7 июля 1978,Мужчина,Москва,"[{'year': 2002, 'name': 'Таганрогский Радиотех...",Руководитель проектов,Высшее образование,NaN,NaN,Руководитель проектов,Работоспособность,"Являюсь разносторонним человеком, люблю хорошо...",7554.0,"Организация рабочего процесса на объекте , вза..."
2,20 января 1986,Мужчина,Москва,"[{'year': 2012, 'name': 'Институт Проблем Риск...",Руководитель проекта,Высшее образование,NaN,NaN,Руководитель проектов,"Автоматизация процессов,Проектное управление,О...","Системное мышление, глубокая аналитика и деком...",3732.0,Формирование бизнес требований. Сбор информац...


In [7]:
stemmer = SnowballStemmer('russian')

def clear_txt(txt):
    txt = txt.lower()
    txt = re.sub('[/+_!@#$0-9\n.,:()""«»;-]', ' ', txt)
    new_txt = ''
    for t in txt.split(' '):
        if len(t) > 0:
            new_txt = new_txt + stemmer.stem(t) + ' '
    return new_txt[:-1]

In [6]:
df['skills_desc'] = df['skills_desc'].fillna(' ')
df['exp_description'] = df['exp_description'].fillna(' ')

In [8]:
%%time

for i in range(len(df)):
    
    s1 = re.sub('[^A-Za-zА-Яа-я]+', ' ', df.loc[i,'skills_desc'])
    s2 = re.sub('[^A-Za-zА-Яа-я]+', ' ', df.loc[i,'exp_description'])
    
    df.loc[i, 'skills_desc_stem'] = clear_txt(s1)
    df.loc[i, 'exp_desc_stem'] = clear_txt(s2)
    
    if i%1000==0:
        print('обработано', i)

обработано 0
обработано 1000
обработано 2000
обработано 3000
обработано 4000
обработано 5000
обработано 6000
обработано 7000
обработано 8000
обработано 9000
Wall time: 3min 6s


In [20]:
df.tail()

,birth_date,gender,area,education,title,education_level,salary.amount,salary.currency,specialization,skill_set,skills_desc,exp_length,exp_description,skills_desc_stem,exp_desc_stem
9098,22 ноября 1991,Женщина,Москва,"[{'year': 2011, 'name': 'Волгоградский институ...",Начальник службы безопасности,Среднее специальное образование,50000.0,руб.,Специалист службы безопасности,"Деловое общение,Творческое мышление,Работа в к...","быстрая обучаемость, стрессоустойчивость, комм...",1643.0,Проверка работы сотрудников охраныОтветственно...,быстр обучаем стрессоустойчив коммуникабельн,проверк работ сотрудник охраныответствен стрес...
9099,18 марта 1996,Мужчина,Москва,"[{'year': 2021, 'name': 'НИУ СГУ им. Чернышевс...",Начинающий специалист,Высшее образование,90000.0,руб.,Специалист службы безопасности,Исполнительность,Прохождение срочной службы в в/ч 06017 на долж...,2118.0,Предотвращение от актов незаконного вмешательс...,прохожден срочн служб в в ч на должност замест...,предотвращен от акт незакон вмешательств на ж ...
9100,3 октября 1976,Мужчина,Москва,"[{'year': 2007, 'name': 'Московская финансово-...",Руководитель службы безопасности/старший специ...,Высшее образование,NaN,NaN,Специалист службы безопасности,"Сбор и анализ информации,Работа с большим объе...","Женат, вдое детей, люблю спорт",9331.0,"- Решение вопросов внутренней, внешней, эконом...",женат вдо дет любл спорт,решен вопрос внутрен внешн экономическ и инфор...
9101,27 декабря 1995,Мужчина,Москва,"[{'year': 2017, 'name': 'Московский авиационны...",Заместитель начальника службы безопасности,Высшее образование (Бакалавр),150000.0,руб.,Специалист службы безопасности,"Водительское удостоверение категории B,Обслужи...",,3336.0,Обеспечение безопасности деятельности предприя...,,обеспечен безопасн деятельн предприят в т ч со...
9102,24 мая 1969,Женщина,Москва,"[{'year': 1988, 'name': 'Индустриальный техник...",Помощник по хозяйству (семейная пара),Среднее специальное образование,150000.0,руб.,Полицейский,"Работа с людьми,Управление персоналом,Умение р...","Без вредный привычек, умение быть тактичными, ...",9958.0,"Уборка, стирка, VIP-гардероб, закупка продукто...",без вредн привычек умен быт тактичн чувств юмо...,уборк стирк гардероб закупк продукт оплат счет...


In [9]:
df.columns

Index(['birth_date', 'gender', 'area', 'education', 'title', 'education_level',
       'salary.amount', 'salary.currency', 'specialization', 'skill_set',
       'skills_desc', 'exp_length', 'exp_description', 'skills_desc_stem',
       'exp_desc_stem'],
      dtype='object')

In [10]:
df.to_csv("resumes_step2_stems.csv", encoding = 'utf-16')